In [1]:
import fitz
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from doctr.models import ocr_predictor
from doctr.io import DocumentFile
from PIL import Image
import torch
from transformers import VisionEncoderDecoderModel, TrOCRProcessor,BitsAndBytesConfig
from pdf2image import convert_from_path
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sklearn.preprocessing import StandardScaler
import torch
import json
import csv
import threading
import requests
# import response
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0
    )

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device

'cuda'

In [4]:
torch.cuda.is_available()

True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
detector = ocr_predictor(pretrained=True, assume_straight_pages=True).to(device)
trocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten").to(device)
trocr_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
trocr_model.eval()

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-23): 24 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=False)
              (key): Linear(in_features=1024, out_features=1024, bias=False)
              (value): Linear(in_features=1024, out_features=1024, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=1024, out_features=4096, bias=True)
    

In [5]:
def classify_and_extract_pdf(pdf_path, text_threshold=20):
    doc = fitz.open(pdf_path)
    image_pages = 0
    text_pages = 0
    full_text = ""

    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()
        images = page.get_images(full=True)

        if len(text.strip()) < text_threshold and len(images) > 0:
            image_pages += 1
        else:
            text_pages += 1
            full_text += f"\n--- Page {page_num + 1} ---\n{text}"

    doc.close()

    if image_pages > text_pages:
        return 0
    else:
        return full_text.strip()

In [6]:
def preprocess_image(pil_image):
    """Convert a PIL image to grayscale and apply adaptive thresholding."""
    img = np.array(pil_image)  
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 10)
    return Image.fromarray(binary).convert("RGB")  
def cluster_lines(boxes, y_threshold=15):
    """Groups bounding boxes into lines and sorts them correctly."""
    boxes = sorted(boxes, key=lambda b: b[1])
    lines = []
    current_line = []
    for box in boxes:
        if not current_line:
            current_line.append(box)
        else:
            prev_box = current_line[-1]
            if abs(box[1] - prev_box[1]) < y_threshold:  
                current_line.append(box)
            else:
                lines.append(sorted(current_line, key=lambda b: b[0]))
                current_line = [box]

    if current_line:
        lines.append(sorted(current_line, key=lambda b: b[0]))
    return lines  
def process_pdf(pdf_path,trocr_model,detector):
    """Extract text from a PDF, visualize detected text, and save results."""
    output_folder = "extracted_data"
    os.makedirs(output_folder, exist_ok=True)

    pdf_name = os.path.basename(pdf_path)  
    txt_filename = os.path.splitext(pdf_name)[0] + ".txt"
    txt_filepath = os.path.join(output_folder, txt_filename)

    images = convert_from_path(pdf_path, dpi=300)  
    extracted_text = []

    for page_num, img in enumerate(images, start=1):
        print(f"Processing Page {page_num}...")

        temp_image_path = f"temp_page_{page_num}.png"
        img.save(temp_image_path)

        single_page_doc = DocumentFile.from_images(temp_image_path)
        result = detector(single_page_doc)

        img_width, img_height = img.size  

        bounding_boxes = []
        for page in result.pages:
            for block in page.blocks:
                for line in block.lines:
                    (x_min, y_min), (x_max, y_max) = line.geometry
                    left = int(x_min * img_width)
                    top = int(y_min * img_height)
                    right = int(x_max * img_width)
                    bottom = int(y_max * img_height)
                    bounding_boxes.append((left, top, right, bottom))

        sorted_lines = cluster_lines(bounding_boxes)

        cropped_lines, line_positions = [], []
        for line_boxes in sorted_lines:
            left = min(box[0] for box in line_boxes)
            top = min(box[1] for box in line_boxes)
            right = max(box[2] for box in line_boxes)
            bottom = max(box[3] for box in line_boxes)

            crop = img.crop((left, top, right, bottom))
            processed_crop = preprocess_image(crop)
            cropped_lines.append(processed_crop)
            line_positions.append((left, top, right, bottom))

        if cropped_lines:
            batch_pixel_values = trocr_processor(images=cropped_lines, return_tensors="pt").pixel_values.to(device)
            with torch.no_grad():
                generated_ids = trocr_model.generate(batch_pixel_values)
                predictions = trocr_processor.batch_decode(generated_ids, skip_special_tokens=True)
        else:
            predictions = []

        extracted_text.append(f"Page {page_num}:\n" + "\n".join(predictions) + "\n\n")

        fig, ax = plt.subplots(figsize=(12, 12))
        ax.imshow(img)

        for (left, top, right, bottom), text in zip(line_positions, predictions):
            rect = patches.Rectangle((left, top), right - left, bottom - top, linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            ax.text(left, top - 5, text, fontsize=8, color='blue', bbox=dict(facecolor='white', alpha=0.5))

        plt.axis('off')
        plt.show()

        os.remove(temp_image_path)
    txt=""
    for i in extracted_text:
        txt+=i
        txt+='\n'
    return txt

# In case any update in embeddings

In [7]:
paths=[
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/1.Contractual Document.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/2.Regulatory and Compliance Sub Categories.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/3.IPR Categories.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/4.Litigation and Dispute Categories.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/5.Property and Real Estate.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/6.Corporate and Governance Categories.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/7.HR Document categories.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/8.Finance and Banking Categories.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/9.Government and public laws.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/10.Personal and Civil Legal Documents.txt",
    "/home/pa/ml/Document-Classification/CLASSIFICATION FILES/11.Bill of Entry.txt"
]
emb_dict={}
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0
    )
model=HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
    model_kwargs={
        "device": device,
    }
)
for i in paths:
    name=i.split('/')[-1].split('.')[1]
    print("Processing",name,"Document")
    with open(i,'r') as file:
        new_context=file.read()
    docs=text_splitter.split_text(new_context)
    docs=model.embed_documents(docs)
    doc=np.zeros(len(docs[0]))
    for j in docs:
        doc+=j
    emb_dict[i]=doc


Processing Contractual Document Document
Processing Regulatory and Compliance Sub Categories Document
Processing IPR Categories Document
Processing Litigation and Dispute Categories Document
Processing Property and Real Estate Document
Processing Corporate and Governance Categories Document
Processing HR Document categories Document
Processing Finance and Banking Categories Document
Processing Government and public laws Document
Processing Personal and Civil Legal Documents Document
Processing Bill of Entry Document


In [11]:
import requests
import numpy as np
CORE = "emb_core"
SOLR_URL = f"http://localhost:8983/solr/{CORE}/update?commit=true"
HEADERS  = {"Content-Type": "application/json"}

def index_emb_dict(emb_dict):
    docs = []
    for path, arr in emb_dict.items():
        vec = arr.tolist()
        docs.append({
            "id": path,           
            "filepath": path,
            "total_embedding": vec
        })
    print(len(docs))
    resp = requests.post(SOLR_URL, headers=HEADERS, json=docs)
    if resp.ok:
        print(f"[OK] Indexed {len(docs)} documents.")
    else:
        print(f"[FAIL] {resp.status_code}\n{resp.text}")
index_emb_dict(emb_dict)

11
[OK] Indexed 11 documents.


# continue

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0
    )
model=HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
)
CORE     = "emb_core"
BASE_URL = f"http://localhost:8983/solr/{CORE}/select"
resp0 = requests.get(BASE_URL, params={"q": "*:*", "wt": "json", "rows": 0})
resp0.raise_for_status()
total = resp0.json()["response"]["numFound"]
resp1 = requests.get(
    BASE_URL,
    params={"q": "*:*", "wt": "json", "rows": total}
)
resp1.raise_for_status()
docs = resp1.json()["response"]["docs"]
emb_dict = {}
for d in docs:
    path = d["id"]
    vec  = d.get("total_embedding", [])
    emb_dict[path] = np.array(vec, dtype=float)
emb_dict.pop("/home/pa/ml/Document-Classification/CLASSIFICATION FILES/11.Bill of Entry.txt")
print(f" emb_dict having {len(emb_dict)} entries.")


 emb_dict having 10 entries.


In [14]:
def classification(text,emb_dict,name):
    docs1=text_splitter.split_text(text)
    for i in range(len(docs1)):
        docs1[i]+=" "+name
    docs1=model.embed_documents(docs1)
    doc1=np.zeros(len(docs1[0]))
    for k in docs1:
        doc1+=k
    file=""
    csin1=0
    for i in emb_dict:
        name=i.split('/')[-1].split('.')[1]
        # print(name)
        csin=cosine_similarity([emb_dict[i]],[doc1])
        # print(name)
        # print(csin)
        if csin>csin1:
            csin1=csin
            file=name
    return file,docs1,doc1

In [2]:
# below is function to send result after classification to solar

In [16]:
SOLR_URL = "http://localhost:8983/solr/mycore/update/json/docs?commit=true"
HEADERS  = {"Content-Type": "application/json"}

def send_result_to_solr(result: dict):
    """
    Sends one document (result dict) directly to Solr.
    """
    for field in ("section_embedding", "total_embedding"):
        val = result.get(field)
        if hasattr(val, "tolist"):
            result[field] = val.tolist()

    resp = requests.post(SOLR_URL, headers=HEADERS, json=result)
    if resp.status_code == 200:
        print(f"[OK] Indexed id={result.get('id')}")
    else:
        print(f"[FAIL] id={result.get('id')} → {resp.status_code}\n{resp.text}")

In [17]:
# below function will read all files from given folder , classify it and then store it into solar db

In [22]:
def main(folder_path="/home/pa/Downloads/Marg Invoices/(8-4-2025)/"):
    paths=os.listdir(folder_path)
    for path in paths:
        if not path.lower().endswith(".pdf"):
            continue 
        name=path.split("\\")[-1].split('.')[0]
        os.makedirs("/home/pa/ml/tf_gpu/OCR/D:/BPL/BLS/Result/", exist_ok=True)
        output_json=f"/home/pa/ml/tf_gpu/OCR/D:/BPL/BLS/Result/{name}.json"
        ppath=os.path.join(folder_path,path)
        # print(classify_and_extract_pdf(ppath))
        text1=""
        results=[]
        
        text1+=name;
        text1+=" "
        out=classify_and_extract_pdf(ppath)
        # print(out)
        if(out==0):
            print(f"{name} Document seems to be a pdf of image")
            text1=process_pdf(ppath,trocr_model,detector)
        else:
            print(f"{name} Document is pdf of text")
            text1=out
        class1,docs1,doc1=classification(text1,emb_dict,name)
        print("Document best classified as",class1)
        # print(text1)
        # break
        result = {
            "id":name,
            "class": class1,
            "text": text1, 
            "section_embedding": docs1, 
            "total_embedding": doc1.tolist() if hasattr(doc1, "tolist") else doc1 
        }
        send_result_to_solr(result)
        print()
        print()

main("/home/pa/ml/tf_gpu/OCR/D:/BPL/BLS/DownloadDoc/")

Anil_Surendrasingh_Yadav_vs_State_Of_Gujarat_on_27_December_2019 Document is pdf of text
Document best classified as Litigation and Dispute Categories
[OK] Indexed id=Anil_Surendrasingh_Yadav_vs_State_Of_Gujarat_on_27_December_2019


ALCOSTORESINC_12_14_2005-EX-10 Document is pdf of text
Document best classified as Contractual Document
[OK] Indexed id=ALCOSTORESINC_12_14_2005-EX-10


US7054127 Document is pdf of text
Document best classified as IPR Categories
[OK] Indexed id=US7054127




In [23]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0
    )

In [24]:
def embedding_conversion(li,embedding_dim=1024):
    flat_embedding=li
    if len(flat_embedding) % embedding_dim != 0:
        raise ValueError("Flat embedding length is not divisible by embedding dimension.")
    
    num_vectors = len(flat_embedding) // embedding_dim
    section_embedding_2d = [
        flat_embedding[i * embedding_dim: (i + 1) * embedding_dim]
        for i in range(num_vectors)
    ]
    return section_embedding_2d

In [11]:
# below cell is to get data from solar

In [25]:
core_name = "mycore"
solr_url = f"http://localhost:8983/solr/{core_name}/select"
initial_params = {
    "q": "*:*",
    "wt": "json",
    "rows": 0
}
response = requests.get(solr_url, params=initial_params)
total_docs = response.json()["response"]["numFound"]
params = {
    "q": "*:*",
    "wt": "json",
    "rows": total_docs
}
response = requests.get(solr_url, params=params)
docs = response.json()["response"]["docs"]
dic={}
docs1=docs
for i in range(len(docs)):
    li=[docs[i]['class'][0],docs[i]['text'][0],embedding_conversion(docs[i]["section_embedding"]),docs[i]['total_embedding']]
    dic[i]=li

In [26]:
import faiss
from sklearn.metrics.pairwise import euclidean_distances,cosine_similarity
model=HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
)
def context_provider(dic,query):
    idx=''
    query=model.embed_query(query)
    query_emb=np.array([query],dtype='float32')
    sim=0
    for i in dic:
        tot_emb=np.array([dic[i][-1]])

        cos_sim = cosine_similarity(query_emb, tot_emb)
        print(cos_sim)
        if(cos_sim>sim):
            sim=cos_sim
            idx=i
    text,sec_emb=dic[idx][1],np.array(dic[idx][2],dtype='float32')
    faiss.normalize_L2(sec_emb)
    db = faiss.IndexFlatIP(sec_emb.shape[1])
    db.add(sec_emb)
    D, I = db.search(query_emb, k=8)
    context=""
    text_split=text_splitter.split_text(text)
    # print(I)
    for i in I[0]:
        context+=text_split[i]
        context+='\n'
    return context

In [ ]:
import requests

core_name = "mycore"
url = f"http://localhost:8983/solr/{core_name}/update?commit=true"
headers = {"Content-Type": "application/json"}
payload = {"delete": {"query": "*:*"}}

response = requests.post(url, headers=headers, json=payload)
print("Deleted all docs:", response.status_code, response.text)


Deleted all docs: 200 {
  "responseHeader":{
    "status":0,
    "QTime":15
  }
}


In [27]:
query=input("What is your question:: ")
print(context_provider(dic,query))

[[0.82558457]]
[[0.72662516]]
[[0.73071886]]
the case rests on circumstantial evidence has no role to play. It was observed in paragraph 40 as
under:-
Anil Surendrasingh Yadav vs State Of Gujarat on 27 December, 2019
Indian Kanoon - http://indiankanoon.org/doc/137212109/
25
Indian Kanoon - http://indiankanoon.org/doc/137212109/
9
--- Page 1 ---
Anil Surendrasingh Yadav vs State Of Gujarat on 27 December,
2019
Equivalent citations: AIRONLINE 2019 GUJ 853
Author: Bela M. Trivedi
Bench: Bela M. Trivedi, A.C. Rao
       R/CR.A/1973/2019                                        CAV JUDGMENT
            IN THE HIGH COURT OF GUJARAT AT AHMEDABAD
                     R/CRIMINAL APPEAL NO. 1973 of 2019
                                With
              R/CRIMINAL CONFIRMATION CASE NO. 2 of 2019
FOR APPROVAL AND SIGNATURE:
HONOURABLE MS.JUSTICE BELA M. TRIVEDI                           Sd/-
and
HONOURABLE MR.JUSTICE A.C. RAO                                  Sd/-
1     Whether Reporters of Local Pa

In [1]:
!pip freeze > requirements.txt

# DMS Phase 2

In [1]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
import requests
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0
    )
import faiss
from sklearn.metrics.pairwise import euclidean_distances,cosine_similarity
from langchain_huggingface import HuggingFaceEmbeddings
model=HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
)

/home/pa/miniconda3/envs/inf_engine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [65]:
import json
def inference(prompt, stream=False):
    url = "http://10.111.1.62:1147/infer_stream"
    payload = {
        "prompt": f"{prompt}",
        "max_gpu1_memory": "8GB",
        "max_gpu2_memory": "8GB",
        "model": "meta-llama/Llama-3.1-8B-Instruct",
    }
    text = ""
    with requests.post(url, json=payload, stream=True) as r:
        for line in r.iter_lines():
            if line:
                if stream:
                    print(line.decode("utf-8"))
                text += line.decode("utf-8")
    return text

                
def chatprompts(flag,inp,query):
    if(flag==1):
        len1=len(inp)
        context=""
        for i in range(len1):
            context+=f"Document {i+1}:"
            context+=inp[i]
            context+='/n'
        message=[
            {
                'role':'system',
                'content':f"""
                You are a helper which helps user based on the context you recieve
                {context}
                """
            },
            {
                "role":'user',
                'content':f"query {query}"
            }
        ]
        message=tokenizer.apply_chat_template(message,tokenize=False)
        return message
    else:
        message=[
            {
                'role':'system',
                'content':f"""
                You are a helper who help user by responding back to their query
                """
            },
            {
                "role":'user',
                'content':f"query {query}"
            }
        ]
        return message
def embedding_conversion(li,embedding_dim=1024):
    flat_embedding=li
    if len(flat_embedding) % embedding_dim != 0:
        raise ValueError("Flat embedding length is not divisible by embedding dimension.")
    
    num_vectors = len(flat_embedding) // embedding_dim
    section_embedding_2d = [
        flat_embedding[i * embedding_dim: (i + 1) * embedding_dim]
        for i in range(num_vectors)
    ]
    return section_embedding_2d
def load_docs():
    core_name = "mycore"
    solr_url = f"http://10.111.65.43:8983/solr/{core_name}/select"
    initial_params = {
        "q": "*:*",
        "wt": "json",
        "rows": 0
    }
    response = requests.get(solr_url, params=initial_params)
    total_docs = response.json()["response"]["numFound"]
    params = {
        "q": "*:*",
        "wt": "json",
        "rows": total_docs
    }
    response = requests.get(solr_url, params=params)
    docs = response.json()["response"]["docs"]
    dic={}
    docs1=docs
    for i in range(len(docs)):
        li=[docs[i]['class'][0],docs[i]['text'][0],embedding_conversion(docs[i]["section_embedding"]),docs[i]['total_embedding']]
        dic[i]=li
    return dic
def context_provider(query):
    idx=''
    query=model.embed_query(query)
    query_emb=np.array([query],dtype='float32')
    sim=0
    dic=load_docs()
    for i in dic:
        tot_emb=np.array([dic[i][-1]])

        cos_sim = cosine_similarity(query_emb, tot_emb)
        # print(cos_sim)
        if(cos_sim>sim):
            sim=cos_sim
            idx=i
    text,sec_emb=dic[idx][1],np.array(dic[idx][2],dtype='float32')
    faiss.normalize_L2(sec_emb)
    db = faiss.IndexFlatIP(sec_emb.shape[1])
    db.add(sec_emb)
    D, I = db.search(query_emb, k=6)
    context=""
    text_split=text_splitter.split_text(text)
    # print(I)
    
    for i in I[0]:
        context+=text_split[i]
        context+='\n'
    context+= f"type:{dic[idx][0]}"
    return context

# query = "what is similarity between judgment by hc of chattisgarh and hr of bpl on attendence"
def chat(query):
    messages = [
        {
            "role": "system",
            "content": """
    You are a DMS assistant.

    Your ONLY task is to output a JSON object in the following format:

    { "tool": "compare_documents", "query": ["<doc1>", "<doc2>", ...] }

    Rules:
    - "IF THERE IS MENTION OF TWO DIFFERENT DOCUMENTS THEN ITS COMPARE DOCUMENT TOOL"
    - "tool" must always be "compare_documents".
    - "query" must always be a LIST of document references mentioned by the user.
    - Extract document references cleanly (e.g., "document 1" → "doc_1").
    - Never return a single string inside "query".
    - Never add extra text or explanations outside the JSON.

    Example:

    User: "What is similarity between document 1 and document 2?"
    Assistant: { "tool": "compare_documents", "query": ["document 1", "document 2"] }

    User: "What is the similarity between judgment by hc of chattisgarh and hr of bpl on attendence?"
    Assistant: { "tool": "compare_documents", "query": ["judgment by hc of chattisgarh", "hr of bpl on attendence document"] }
    
    User: "What is the case of madhu vyas and then tell me about case of stray dogs"
    Assistant: { "tool": "compare_documents", "query": ["what is case of madhu vyas", "what is case of stray dogs"] }
    """
        },
        {
            "role": "user",
            "content": f"{query}"
        }
    ]
    
    messages = [
        {
            "role": "system",
            "content": """
    You are a DMS assistant.

    Your ONLY task is to output a JSON object in the following format:

    {
        "tool": "<compare_documents | single_document>",
        "query": ["<doc1>", "<doc2>", ...]
    }

    Rules:
    - If there is mention of TWO OR MORE different documents → use "compare_documents".
    - If there is mention of ONLY ONE document → use "single_document".
    - "query" must always be a LIST of document references mentioned by the user.
    - Extract document references cleanly (e.g., "document 1" → "doc_1").
    - Never return a single string inside "query", it must always be a list.
    - Never add extra text or explanations outside the JSON.

    Examples:

    User: "What is similarity between document 1 and document 2?"
    Assistant: { "tool": "compare_documents", "query": ["document 1", "document 2"] }

    User: "What is the similarity between judgment by hc of chattisgarh and hr of bpl on attendence?"
    Assistant: { "tool": "compare_documents", "query": ["judgment by hc of chattisgarh", "hr of bpl on attendence document"] }

    User: "What is the case of madhu vyas?"
    Assistant: { "tool": "single_document", "query": ["case of madhu vyas"] }

    User: "Tell me about the case of stray dogs"
    Assistant: { "tool": "single_document", "query": ["case of stray dogs"] }

    User: "What is the case of madhu vyas and then tell me about case of stray dogs"
    Assistant: { "tool": "compare_documents", "query": ["case of madhu vyas", "case of stray dogs"] }
    """
        },
        {
            "role": "user",
            "content": f"{query}"
        }
    ]


    prompt=tokenizer.apply_chat_template(messages,tokenize=False)
    out=inference(prompt,True)
    out1=json.loads(out[9:-12])
    if out1['tool']=='compare_documents':
        contexts=[]
        if len(out1['query'])==1:
            contexts.append(context_provider(query))
        else:
            for i in out1['query']:
                contexts.append(context_provider(i))
        return (contexts,1)
    if out1['tool']=='single_document':
        contexts=[]
        if len(out1['query'])==1:
            contexts.append(context_provider(query))
            return (contexts,2)
    # return out
query = "tell me about procedures to be follow in court document and is it a litigation and dispute kind of document then tell me about madhu vyas case "
out=chat(query)
if(out[-1]==1):
    prompt=chatprompts(out[1],out[0],query)
    inference(prompt,True)
else:
    prompt=chatprompts(1,out[0],query)
    inference(prompt,True)
# print(prompt)

assistant
{
  "tool": "compare_documents",
  "query": ["procedures to be follow in court document", "madhu vyas case"]
}data: [DONE]
assistant
**Procedures to be followed in a Court Document:**
The procedures to be followed in a court document are as follows:
1.  **Filing of the Case**: The first step is to file the case in the court by submitting the necessary documents and papers.
2.  **Issuance of Summons**: After the case is filed, the court issues a summons to the parties involved, requiring them to appear in court.
3.  **Appearance of Parties**: The parties involved appear in court and present their arguments and evidence.
4.  **Hearing of the Case**: The court hears the arguments and evidence presented by both parties and makes a decision.
5.  **Judgment**: The court delivers its judgment, which is the final decision in the case.
6.  **Appeal**: If either party is not satisfied with the judgment, they can appeal to a higher court.
**Is it a Litigation and Dispute kind of documen

In [62]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helper which helps user based on the context you recieve
                Document 1:type:Litigation and Dispute Categories--- Page 9 ---
Section 78(2) provides that the Court issuing a warrant under subsection (1) shall forward, along
with the warrant, the substance of the information against the person to be arrested, together with
such documents, if any, as maybe sufficient to enable the Court acting under Section 81, to decide
whether bail should or should not be granted to the person.
Section 79 provides for the procedure to be adopted for the execution of warrant by the Executive
Magistrate or by Police officer, not below the rank of an officer in charge of a Police Station, and
Section 80 provides for procedure of arrest of the person for whom the warrant has been issued.
13. Sri Nadeem Murtaza has argued that different Benches of this Court follow

In [11]:
tokenizer.apply_chat_template(prompt,tokenize=False)

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a helper which helps user based on the context you recieve\n                Document 1:--- Page 4 ---\n4\nrecorded owner and have been in peaceful, undisputed possession since \nmutation in 2022. Despite filing proper applications and completing all \nstatutory  formalities,  the  revenue  officials  have  not  acted  upon  the \npetitioners' request and kept the matter pending without any written \norder  or  explanation.  This  inaction  is  in  clear  violation  of  the \nChhattisgarh Lok Sewa Guarantee Adhiniyam, 2011, which mandates \ntime-bound delivery of public services including issuance of land-related \ncertificates.  The  conduct  of  the  authorities  violates  the  petitioners' \nconstitutional  rights  under  Articles  14  and  300A,  amounts  to \nadministrative arbitrariness, and constitutes a failure of public duty. The \npetitioner

In [18]:
print(context_provider(dic,"judgement by supreme court of india on stray dogs"))

[[0.76343728]]
[[0.76269105]]
[[0.73689712]]
[[0.77397857]]
[[0.82529768]]
[[0.8334111]]
[[0.84142747]]
[[0.87094527]]
--- Page 2 ---
sterilization, is not the proper solution in the present matter, due to the large number of dogs
involved in the present case.
5. This Court notes that various instances of dog bites by stray dogs have come forth, which are
reported regularly in the newspapers, along with several petitions before this Court, wherein,
instances of dog bites have been brought to the notice of this Court.
6. This Court in the case of Geeta Devi Versus Govt. of NCT of Delhi and Others, 2024 SCC OnLine
Del 7064, dealt with a writ petition which was filed seeking compensation of Rs. 50 Lacs, emanating
from an unfortunate incident, wherein, a 5-month-old infant child was fatally bitten by a stray dog
which led to the death of the child. This Court, while dealing with the said case, observed numerous
instances being dealt with by the High Courts all across the country, in relati